## Analyzing Most Common Names for Kids in America

In [1]:
# import modules
import pandas as pd
import sqlite3 as sql

### Connecting to sql database

In [2]:
# connecting to database
# new babynames_db will be created if not already present
db_connection = sql.connect('babynames_db.db')

In [3]:
# creating a cursor on the db connection using cursor method
execute_sql = db_connection.cursor()

### Importing the table as a dataframe

In [4]:
#import the table as a dataframe
names = pd.read_csv("C:\\Users\\91740\\Desktop\\SQL\\Exploratory Data Analysis with SQL\\US BabyNames Analysis\\NationalNames.csv")
names.head(2)

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604


### Insert the table in sql database

In [5]:
# writng the dataframe into sql database
# using pandas.DataFrame.to_sql method
# help(names.to_sql)
names.to_sql('baby_names',
             con = db_connection,
             if_exists = 'replace')

1825433

### Table schema - pd.read_sql method

In [6]:
# execute sql on the table using pandas.read_sql method
# query top 5 from the table
# sql and con are mandetory arguments
# for sql lite instead of top 5 use limit 5
top_five = pd.read_sql(sql = """
                            SELECT *
                            FROM baby_names
                            LIMIT 5;
                            """,
                       con = db_connection)
top_five

,index,Id,Name,Year,Gender,Count
0,0,1,Mary,1880,F,7065
1,1,2,Anna,1880,F,2604
2,2,3,Emma,1880,F,2003
3,3,4,Elizabeth,1880,F,1939
4,4,5,Minnie,1880,F,1746


### Question 1

#### Most popular names of all time?

In [7]:
# most popular names in all time?
# identify distinct names and there count ?
distinct_names = pd.read_sql(sql = """ SELECT DISTINCT Name,
                                        SUM(Count) AS Names_Frequency
                                        FROM baby_names
                                        GROUP BY Name
                                        ORDER BY Names_Frequency DESC
                                        """,
                             con = db_connection)
distinct_names

,Name,Names_Frequency
0,James,5129096
1,John,5106590
2,Robert,4816785
3,Michael,4330805
4,Mary,4130441
...,...,...
93884,Aadrit,5
93885,Aadrian,5
93886,Aadian,5
93887,Aadhyan,5


#### Export Result as csv

In [9]:
# help(names.to_csv)
# type(distinct_names)
# export the distinct_names dataframe into a cvs file
distinct_names.to_csv('C:\\Users\\91740\\Desktop\\SQL\\Exploratory Data Analysis with SQL\\US BabyNames Analysis\\result_distinct_names.csv',
                      index = True,
                     index_label = 'S.no')

### Question 2

#### Most popular male and female names?

In [10]:
# ?? most popular male and female names?
# number of baby names by gender ?
names_by_gender = pd.read_sql("""SELECT Name, Gender, 
                                SUM(CASE WHEN Gender = 'F' THEN Count
                                ELSE 0 END) as F_Frequency,
                                SUM(CASE WHEN Gender = 'M' THEN Count
                                ELSE 0 END) as M_Frequency
                                FROM baby_names
                                GROUP BY Name, Gender
                                ORDER BY SUM(Count) DESC
                                """,
                              con = db_connection)
names_by_gender           

,Name,Gender,F_Frequency,M_Frequency
0,James,M,0,5105919
1,John,M,0,5084943
2,Robert,M,0,4796695
3,Michael,M,0,4309198
4,Mary,F,4115282,0
...,...,...,...,...
104105,Zysean,M,0,5
104106,Zytaevius,M,0,5
104107,Zytavion,M,0,5
104108,Zyvion,M,0,5


#### Export result as csv

In [11]:
# export into csv file
names_by_gender.to_csv('C:\\Users\\91740\\Desktop\\SQL\\Exploratory Data Analysis with SQL\\US BabyNames Analysis\\result_names_by_gender.csv',
                      index = True,
                     index_label = 'S.no')

### Question 3

#### Year wise most popular names?

In [12]:
# ?? popular names year wise
#    --year as header and popular name as row value
popular_names_year = pd.read_sql("""WITH t1 as
                                    (
                                    SELECT Name,Year,
                                    SUM(Count) as Frequency,
                                    rank() over(partition by Year order by SUM(Count) desc) as name_rank
                                    FROM baby_names
                                    GROUP BY Name,Year
                                    ),t2 as
                                    (
                                    select * from t1
                                    where name_rank = 1
                                    )
                                    select *
                                    from t2
                                    """,
                                 con = db_connection)
popular_names_year
# cant write a pivot function here, sql lite does not support pivot function
# writing case statement on 132 records is not practical hence, can derive thos result on tableau

,Name,Year,Frequency,name_rank
0,John,1880,9701,1
1,John,1881,8795,1
2,John,1882,9597,1
3,John,1883,8934,1
4,John,1884,9428,1
...,...,...,...,...
130,Isabella,2010,22913,1
131,Sophia,2011,21839,1
132,Sophia,2012,22285,1
133,Sophia,2013,21172,1


#### Export result as csv

In [13]:
## export the result into csv file 
popular_names_year.to_csv('C:\\Users\\91740\\Desktop\\SQL\\Exploratory Data Analysis with SQL\\US BabyNames Analysis\\result_popular_names_year.csv',
                      index = True,
                     index_label = 'S.no')

### Question 4

#### Which male and female names have been popular for most years?

In [14]:
#?? which male and female names have been popular for most years
famous_names_most_year = pd.read_sql("""WITH t1 as
                                    (
                                    SELECT Name,Year,Gender,
                                    SUM(Count) as Frequency,
                                    rank() over(partition by Year,Gender order by SUM(Count) desc) as name_rank
                                    FROM baby_names
                                    GROUP BY Name,Year,Gender
                                    ),t2 as
                                    (
                                    select * from t1
                                    where name_rank = 1
                                    )
                                    select Name,Gender,count(*) as popularity
                                    from t2
                                    group by Name,Gender
                                    order by popularity desc
                                    """,
                                 con = db_connection)
famous_names_most_year

,Name,Gender,popularity
0,Mary,F,76
1,John,M,44
2,Michael,M,44
3,Robert,M,17
4,Jennifer,F,15
5,Jacob,M,14
6,James,M,13
7,Emily,F,12
8,Jessica,F,9
9,Lisa,F,8


#### Export result as csv

In [16]:
# export result into csv file
famous_names_most_year.to_csv('C:\\Users\\91740\\Desktop\\SQL\\Exploratory Data Analysis with SQL\\US BabyNames Analysis\\result_famous_names_most_year.csv',
                      index = True,
                     index_label = 'S.no')